In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install rouge_score nltk sacrebleu pandas numpy scikit-learn

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-863_olbb/unsloth_3ed105739cd6492c9f29743281f6b0f4
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-863_olbb/unsloth_3ed105739cd6492c9f29743281f6b0f4
  Resolved https://github.com/unslothai/unsloth.git to commit 0fb14e6a76f3695d01314d7b3faf7252141d9f56
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.4/284.4 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 143.4 MB/s eta 0:00:00

In [ ]:
# --- MUST BE IMPORTED FIRST ---
from unsloth import FastLanguageModel

import os
# Fix for memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
import pandas as pd
import numpy as np
import time
import json
import uuid
import logging
import math
import shutil
import re
import gc # Added for garbage collection
from abc import ABC, abstractmethod
from datetime import datetime
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import TrainingArguments, TextStreamer
from trl import SFTTrainer
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from IPython.display import FileLink, display


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# --- 1. CONFIGURATION & LOGGING ---

class Config:
    MODEL_ID = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
    MAX_SEQ_LENGTH = 2048
    DTYPE = None
    LOAD_IN_4BIT = True
    DATASET_PATH = "/kaggle/input/dataset12/BengaliEmpatheticConversationsCorpus .csv"
    OUTPUT_DIR = "outputs"
    LOG_DIR = "logs"
    SYSTEM_PROMPT = """আপনি একজন সহানুভূতিশীল মানসিক স্বাস্থ্য পরামর্শদাতা। ব্যবহারকারীর সমস্যার কথা শুনে বাংলায় সহানুভূতিশীল ও আন্তরিক উত্তর দিন।"""

class ExperimentLogger:
    def __init__(self):
        os.makedirs(Config.LOG_DIR, exist_ok=True)
        self.exp_log_path = os.path.join(Config.LOG_DIR, "LLAMAExperiments.csv")
        self.resp_log_path = os.path.join(Config.LOG_DIR, "GeneratedResponses.csv")
        self.test_resp_log_path = os.path.join(Config.LOG_DIR, "TestSet_Responses.csv")
        self._init_logs()

    def _init_logs(self):
        # Metrics Log
        if not os.path.exists(self.exp_log_path):
            df = pd.DataFrame(columns=["id", "model_name", "lora_config", "train_loss", "val_loss", "metrics", "timestamp"])
            df.to_csv(self.exp_log_path, index=False)

        # Validation Responses Log
        if not os.path.exists(self.resp_log_path):
            df = pd.DataFrame(columns=["id", "experiment_id", "input_text", "response_text", "timestamp"])
            df.to_csv(self.resp_log_path, index=False)

        # Hardcoded Test Set Responses Log (Required Deliverable)
        if not os.path.exists(self.test_resp_log_path):
            df = pd.DataFrame(columns=["id", "experiment_id", "topic", "question", "model_response", "timestamp"])
            df.to_csv(self.test_resp_log_path, index=False)

    def log_experiment(self, exp_data):
        df = pd.DataFrame([exp_data])
        df.to_csv(self.exp_log_path, mode='a', header=False, index=False)

    def log_response(self, resp_data):
        df = pd.DataFrame([resp_data])
        df.to_csv(self.resp_log_path, mode='a', header=False, index=False)

    def log_test_response(self, resp_data):
        df = pd.DataFrame([resp_data])
        df.to_csv(self.test_resp_log_path, mode='a', header=False, index=False)


In [ ]:
# --- 2. DATASET PROCESSOR (OOP) ---

class DatasetProcessor:
    def __init__(self, file_path):
        self.file_path = file_path
        self.tokenizer = None

    def load_and_clean(self):
        print(f"Loading Dataset from {self.file_path}...")

        if not os.path.exists(self.file_path):
            print(f"Error: File not found at {self.file_path}")
            return None

        try:
            if self.file_path.endswith('.xlsx'):
                df = pd.read_excel(self.file_path)
            else:
                df = pd.read_csv(self.file_path)

            print(f"   Loaded {len(df)} rows. Cleaning text...")

            # Optimization: Pre-compile regex for speed
            html_tag_pattern = re.compile(r'<[^>]+>')
            whitespace_pattern = re.compile(r'\s+')

            possible_cols = ['Topics', 'Question-Title', 'Questions', 'Answers']
            for col in possible_cols:
                if col in df.columns:
                    # Optimization: Use apply with compiled regex instead of .str.replace
                    df[col] = df[col].astype(str).apply(
                        lambda x: whitespace_pattern.sub(' ', html_tag_pattern.sub('', x)).strip()
                    )

            print("Dataset loaded and cleaned.")
            return df

        except Exception as e:
            print(f"Error processing file: {e}")
            return None

    def prepare_chat_format(self, df, tokenizer):
        self.tokenizer = tokenizer
        # Using Centralized Config Prompt
        system_prompt = Config.SYSTEM_PROMPT

        formatted_data = []

        for _, row in df.iterrows():
            if 'Questions' not in row: continue

            user_input = f"বিষয়: {row.get('Topics', '')}\nশিরোনাম: {row.get('Question-Title', '')}\nপ্রশ্ন: {row.get('Questions', '')}"
            answer = row.get('Answers', '')

            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input},
                {"role": "assistant", "content": answer}
            ]

            text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)

            formatted_data.append({
                "text": text,
                "topic": row.get('Topics', ''),
                "question": row.get('Questions', ''),
                "ground_truth": answer
            })

        return Dataset.from_pandas(pd.DataFrame(formatted_data))


In [ ]:
# --- 3. TRAINING STRATEGY PATTERN (OOP) ---

class TrainingStrategy(ABC):
    @abstractmethod
    def setup_model(self): pass
    @abstractmethod
    def train(self, train_dataset, eval_dataset): pass

class UnslothStrategy(TrainingStrategy):
    def __init__(self):
        self.model = None
        self.tokenizer = None
        self.experiment_id = str(uuid.uuid4())[:8]

    def setup_model(self):
        major_version, minor_version = torch.cuda.get_device_capability()
        if major_version < 7:
            raise RuntimeError("Error: Use GPU T4 x2 (Session Options > Accelerator).")

        print("Setting up Unsloth LLaMA 3.1 Model (LoRA)...")
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name = Config.MODEL_ID,
            max_seq_length = Config.MAX_SEQ_LENGTH,
            dtype = Config.DTYPE,
            load_in_4bit = Config.LOAD_IN_4BIT,
        )

        model = FastLanguageModel.get_peft_model(
            model,
            r = 16,
            target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
            lora_alpha = 16,
            lora_dropout = 0,
            bias = "none",
            use_gradient_checkpointing = "unsloth",
            random_state = 3407,
        )

        self.model = model
        self.tokenizer = tokenizer
        return model, tokenizer

    def train(self, train_dataset, eval_dataset):
        print("🚀 Starting Training with Unsloth...")
        trainer = SFTTrainer(
            model = self.model,
            tokenizer = self.tokenizer,
            train_dataset = train_dataset,
            eval_dataset = eval_dataset,
            dataset_text_field = "text",
            max_seq_length = Config.MAX_SEQ_LENGTH,
            dataset_num_proc = 2,
            packing = True,
            args = TrainingArguments(
                per_device_train_batch_size = 2,
                per_device_eval_batch_size = 1,
                gradient_accumulation_steps = 4,
                warmup_steps = 5,
                max_steps = 300,
                learning_rate = 2e-4,
                fp16 = not torch.cuda.is_bf16_supported(),
                bf16 = torch.cuda.is_bf16_supported(),
                logging_steps = 10,
                optim = "adamw_8bit",
                weight_decay = 0.01,
                lr_scheduler_type = "linear",
                seed = 3407,
                output_dir = Config.OUTPUT_DIR,
                report_to = "none",
                gradient_checkpointing = True,
                gradient_checkpointing_kwargs={"use_reentrant": False},
            ),
        )
        trainer_stats = trainer.train()
        return trainer, trainer_stats

class LLAMAFineTuner:
    def __init__(self, strategy: TrainingStrategy):
        self.strategy = strategy
        self.dataset_processor = None
        self.logger = ExperimentLogger()

    def run(self):
        # Process Main Dataset
        self.dataset_processor = DatasetProcessor(Config.DATASET_PATH)
        df = self.dataset_processor.load_and_clean()
        if df is None: return

        model, tokenizer = self.strategy.setup_model()
        dataset = self.dataset_processor.prepare_chat_format(df, tokenizer)
        # Use smaller test set for memory safety during eval
        dataset = dataset.train_test_split(test_size=0.01)

        trainer, stats = self.strategy.train(dataset['train'], dataset['test'])

        # Eval & Perplexity
        print("Cleaning up memory before evaluation...")
        torch.cuda.empty_cache()
        gc.collect()

        print("Running Evaluation (Perplexity)...")
        try:
            eval_results = trainer.evaluate()
            val_loss = eval_results.get("eval_loss", "N/A")
            perplexity = math.exp(val_loss) if val_loss != "N/A" else "Infinity"
        except Exception as e:
            print(f"Evaluation failed due to memory: {e}. Calculating simplified perplexity.")
            val_loss = "N/A"
            perplexity = "N/A"

        self.logger.log_experiment({
            "id": self.strategy.experiment_id,
            "model_name": Config.MODEL_ID,
            "lora_config": "r=16, alpha=16",
            "train_loss": stats.training_loss,
            "val_loss": val_loss,
            "metrics": f"Perplexity: {perplexity}",
            "timestamp": datetime.now().isoformat()
        })

        print("Training Completed.")
        return model, tokenizer, self.strategy.experiment_id, dataset


In [ ]:
# --- 4. EVALUATOR ---

class Evaluator:
    def __init__(self, model, tokenizer, experiment_id):
        self.model = model
        self.tokenizer = tokenizer
        self.experiment_id = experiment_id
        self.logger = ExperimentLogger()
        self.rouge = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
        FastLanguageModel.for_inference(self.model)

    def generate_response(self, topic, question):
        # Using Centralized Config Prompt
        system_prompt = Config.SYSTEM_PROMPT

        user_input = f"বিষয়: {topic}\nপ্রশ্ন: {question}"
        messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_input}]

        inputs = self.tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

        # INCREASED max_new_tokens & Added Repetition Penalty
        outputs = self.model.generate(
            input_ids=inputs,
            max_new_tokens=1024,  # Increased to ensure full response
            repetition_penalty=1.2, # Added to stop repeating system prompt
            use_cache=True,
            temperature=0.7,
            top_p=0.9
        )
        response = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

        assistant_response = response.split("assistant")[-1].strip() if "assistant" in response else response

        return assistant_response

    def run_automated_evaluation(self, test_dataset, num_samples=3):
        # BLEU/ROUGE on Main Dataset Test Split (Requires Ground Truth)
        print(f"\n Running BLEU/ROUGE on {num_samples} validation samples...")

        available_samples = len(test_dataset)
        actual_samples = min(available_samples, num_samples)

        indices = np.random.choice(available_samples, actual_samples, replace=False)
        total_metrics = {'BLEU': 0, 'ROUGE-1': 0, 'ROUGE-L': 0}

        for idx in indices:
            data = test_dataset[int(idx)]
            gen_res = self.generate_response(data.get("topic", ""), data.get("question", ""))
            ground_truth = data.get("ground_truth", "")

            # Save to main response log
            self.logger.log_response({
                "id": str(uuid.uuid4())[:8],
                "experiment_id": self.experiment_id,
                "input_text": data.get("question", ""),
                "response_text": gen_res,
                "timestamp": datetime.now().isoformat()
            })

            bleu = sentence_bleu([ground_truth.split()], gen_res.split(), weights=(0.5, 0.5))
            rouge = self.rouge.score(ground_truth, gen_res)

            total_metrics['BLEU'] += bleu
            total_metrics['ROUGE-1'] += rouge['rouge1'].fmeasure
            total_metrics['ROUGE-L'] += rouge['rougeL'].fmeasure

        avg_metrics = {k: v / actual_samples for k, v in total_metrics.items()}
        print(f"\n Final Metrics: {avg_metrics}")
        return avg_metrics

    def generate_hardcoded_samples(self):
        """
        Generates responses for 10 specific samples hardcoded from the user's excel file.
        Satisfies 'Sample model responses on test prompts' without needing extra file upload.
        """
        print(f"\n Generating responses for 10 Hardcoded Test Samples...")

        # 10 Samples manually extracted and PARAPHRASED to ensure they are unique test cases
        test_samples = [
            {"topic": "পারিবারিক দ্বন্দ্ব", "question": "আমার সংসারে শান্তি নেই কারণ আমার মা আর বউয়ের মধ্যে সবসময় ঝগড়া লেগেই থাকে। আগে ছোটখাটো বিষয় নিয়ে হতো, কিন্তু এখন তা মারাত্মক আকার ধারণ করেছে। আমার স্ত্রী বলে মা নাকি খুব ডমিনেটিং, আর মা বলে বউ অলস। একবার বউ মায়ের মুখে তর্ক করেছিল, এরপর থেকে পরিস্থিতি আরও খারাপ। সামান্য কথা কাটাকাটি এখন বড় অশান্তিতে রূপ নেয়। আমি এখন কী করব বুঝতে পারছি না।"},
            {"topic": "পদার্থের অপব্যবহার, আসক্তি", "question": "আমরা সন্তান নেওয়ার কথা ভাবছি, তাই আমি স্মোকিং ছেড়ে দিতে চাই। কিন্তু কাজটা আমার জন্য খুব কঠিন হয়ে দাঁড়িয়েছে। এটা এখন আর শুধু শারীরিক নেশা নয়, মানসিক অভ্যাসে পরিণত হয়েছে। আমি কিছুতেই ধুমপানের চিন্তা মাথা থেকে ঝেড়ে ফেলতে পারছি না। এই নেশা থেকে বের হয়ে আসার উপায় কী?"},
            {"topic": "পারিবারিক দ্বন্দ্ব", "question": "আমি একটা বিষয় পরিবারের কাছে গোপন রেখেছি, কিন্তু এটা নিয়ে খুব গিল্টি ফিল করছি। আমি আমার স্ত্রী বা মাকে এটা বলে কষ্ট দিতে চাই না, আবার নিজের ভেতরে চেপে রাখাও কষ্টকর হয়ে যাচ্ছে। এই গোপনীয়তা আমাকে ভেতর থেকে কুড়ে কুড়ে খাচ্ছে এবং আমি খুব এনজাইটিতে ভুগছি। আমার এখন কী করা উচিত? দয়া করে পরামর্শ দিন।"},
            {"topic": "আচরণগত পরিবর্তন, সামাজিক সম্পর্ক", "question": "আমি আমার সম্পর্কের ব্যাপারে খুব বেশি পজেসিভ হয়ে পড়ছি, আর এর ফলে আমার বন্ধুদের সাথে সম্পর্ক নষ্ট হচ্ছে। আমি বুঝতে পারছি এটা আমার একটা সমস্যা। আমি কীভাবে এই স্বভাবটা পরিবর্তন করতে পারি?"},
            {"topic": "দুশ্চিন্তা", "question": "বেশ কয়েক বছর আগে মাথায় একটা চোট পেয়েছিলাম, এরপর থেকে মনটা সবসময় অস্থির হয়ে থাকে। আমি অকারণে খুব বেশি টেনশন করি। কোনোরকম ওষুধ খাওয়া ছাড়াই কি এই উদ্বেগ বা এনজাইটি কমানো সম্ভব?"},
            {"topic": "বিচ্ছিন্নতা", "question": "সবাই বলে আমি দেখতে সুন্দর, কিন্তু এটাকে তারা আমার চরিত্রের নেতিবাচক দিক হিসেবে তুলে ধরে। আগে কখনও এমন কথা শুনিনি, কিন্তু এখন এইসব মন্তব্য আমাকে খুব ভাবাচ্ছে। আমার মনে হচ্ছে আমি হয়তো সত্যিই অবিবেচক বা স্বার্থপর। এসব কথা আমার সম্পর্ককেও প্রভাবিত করছে এবং আমি মানসিকভাবে খুব আঘাত পাচ্ছি। এই পরিস্থিতি সামলাব কীভাবে?"},
            {"topic": "গার্হস্থ্য সহিংসতা", "question": "আমার গার্লফ্রেন্ড আর আমার একটা ছেলে আছে। কিন্তু সমস্যা হলো ও খুব ইমোশনাল এবং আমাকে সবসময় ছোট করে কথা বলে। সে দাবি করে সে প্রতারণা করছে না, কিন্তু তার আচরণে আমি খুব কষ্ট পাই। সে আমাকে সবসময় নিচু দেখায়। বাচ্চার খাতিরে আমি সম্পর্কটা টিকিয়ে রাখতে চাই, কিন্তু এই পরিস্থিতিতে আমার কী করা উচিত?"},
            {"topic": "পদার্থের অপব্যবহার, আসক্তি", "question": "আমার বাবা প্রচুর মদ্যপান করেন। আমি চাই তিনি এটা বন্ধ করুন। তাঁকে কি এমন কোনো রিহ্যাব বা চিকিৎসা কেন্দ্রে পাঠানো সম্ভব যেখানে তিনি সাহায্য পাবেন? যেমন বেকার অ্যাক্টের মতো কোনো আইনি বা প্রাতিষ্ঠানিক সাহায্য কি নেওয়া যেতে পারে?"},
            {"topic": "হতাশা", "question": "আজকাল আমার কোনো কাজেই মন বসে না, ভীষণ একা লাগে। সারাদিন শুধু বিছানায় শুয়ে থাকি, কারো সাথে কথা বলতে বা মিশতে ইচ্ছে করে না। এই বিষণ্ণতা থেকে মুক্তি পাব কীভাবে?"},
            {"topic": "কর্মক্ষেত্রে চাপ", "question": "অফিসের অতিরিক্ত কাজের চাপে আমি মানসিকভাবে ভেঙে পড়েছি। বস সবসময় কাজের ডেডলাইন নিয়ে প্রেশার দেন এবং অতিরিক্ত কাজ চাপিয়ে দেন। আমি আর এই চাপ নিতে পারছি না, খুব ক্লান্ত লাগে।"}
        ]

        for idx, item in enumerate(test_samples):
            topic = item['topic']
            question = item['question']

            print(f"\n--- Test Case {idx+1} ---")
            print(f"Topic: {topic}")
            print(f"Question: {question[:80]}...") # Printing partial question is fine for log readability

            response = self.generate_response(topic, question)

            # FIXED: Printing FULL response (removed slicing)
            print(f"Model: {response}\n")

            # Log specifically for this test set (Deliverable)
            self.logger.log_test_response({
                "id": str(uuid.uuid4())[:8],
                "experiment_id": self.experiment_id,
                "topic": topic,
                "question": question,
                "model_response": response,
                "timestamp": datetime.now().isoformat()
            })

        print(f"\n Generated 10 responses saved to {self.logger.test_resp_log_path}")


In [ ]:
# --- 5. MAIN EXECUTION ---

if __name__ == "__main__":
    strategy = UnslothStrategy()
    app = LLAMAFineTuner(strategy)

    if os.path.exists(Config.DATASET_PATH):
        # 1. Train
        trained_model, tokenizer, exp_id, full_dataset = app.run()

        # 2. Evaluate (Metrics on Validation Set - Needed for your report)
        evaluator = Evaluator(trained_model, tokenizer, exp_id)
        metrics = evaluator.run_automated_evaluation(full_dataset['test'], num_samples=3)

        # 3. Generate Responses on Hardcoded Test Set (Requirement: 10 Samples)
        evaluator.generate_hardcoded_samples()

        # 4. Zip and Download Deliverables (Logs, Responses)
        print("\n Zipping files for download...")
        shutil.make_archive("submission_files", 'zip', Config.LOG_DIR)

        print(f"All deliverables ready!")
        print("Click below to download the Logs and Responses (Use these for your report):")
        display(FileLink(r'submission_files.zip'))

    else:
        print(f"Dataset not found at {Config.DATASET_PATH}")

Loading Dataset from /content/BengaliEmpatheticConversationsCorpus .csv...
   Loaded 38233 rows. Cleaning text...
Dataset loaded and cleaned.
Setting up Unsloth LLaMA 3.1 Model (LoRA)...
==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

...

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Training Completed.

 Running BLEU/ROUGE on 3 validation samples...


/usr/local/lib/python3.12/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)



 Final Metrics: {'BLEU': 0.05555555555555556, 'ROUGE-1': 0.0, 'ROUGE-L': 0.0}

 Generating responses for 10 Hardcoded Test Samples...

--- Test Case 1 ---
Topic: পারিবারিক দ্বন্দ্ব
Question: আমার সংসারে শান্তি নেই কারণ আমার মা আর বউয়ের মধ্যে সবসময় ঝগড়া লেগেই থাকে। আগে ছো...
Model: আপনি একজন ঘনিষ্ঠ সঙ্গী. আপনার চেয়ে আরও পছন্দসই হতে পারে! আপনি তাদের সাথে আরও কাজ করতে পারেন? এই ফ্যাসিনেট কারণ আপনি মনে করি আপনার স্ত্রী আপনার সংসারে সবচেয়ে সৌভাগ্যবান ব্যক্তি. তাই আপনি আপনার পরিবারের ব্যক্তিদের সাথে একটি সত্যিকারের চুক্তি করতে পারেন এবং কোন প্রতিযোগিতার সৃষ্টি না করে একটি "ঐক্য" এবং "এক-টিম" বানাতে পারেন।


--- Test Case 2 ---
Topic: পদার্থের অপব্যবহার, আসক্তি
Question: আমরা সন্তান নেওয়ার কথা ভাবছি, তাই আমি স্মোকিং ছেড়ে দিতে চাই। কিন্তু কাজটা আমার জ...
Model: আপনি ড্রাগ নেওয়ার বিষয়ে ভাবছেন. এটি আপনার স্বাস্থ্যের জন্য ভাল নয়। আপনার ঘৃণা করতে থাকলে আপনার জীবন দুর্গন্ধ এবং উদ্বেগ হবে।

আপনি যখন আপনার ব্যক্তিগত জীবনের জন্য সঙ্গী খুঁজে পাবেন তখন আপনি একটি গ্রুপে যোগ দিতে পারেন, যেখানে আপনি 

/content/submission_files.zip